<a href="https://colab.research.google.com/github/alexferrero23/twitter-category-nlp/blob/develop/src/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import glob
import re
import io

#### Select your file

In [36]:
from google.colab import files
uploaded = files.upload()

Saving working_df.orc to working_df.orc


In [39]:
import pyarrow.orc 

df = pd.read_orc('working_df.orc')

In [42]:
df.head(5)

,username,created_at,text,lang,label,retweet_count,favorite_count,length
0,RepAdams,2022-03-23 18:53:00,"Madeleine Albright often said, “There is a spe...",en,D,56.0,196.0,117
1,RepAdams,2022-03-22 14:40:16,Judge Ketanji Brown Jackson:✔️ Received a unan...,en,D,13.0,26.0,115
2,RepAdams,2022-03-21 20:59:02,When I Grow Up is a virtual forum designed to ...,en,D,5.0,4.0,117
3,RepAdams,2022-03-21 19:55:53,"Ketanji Brown Jackson's parents, Ellery &amp; ...",en,D,8.0,31.0,120
4,RepAdams,2022-03-21 13:58:54,Judge Ketanji Brown Jackson is the most qualif...,en,D,19.0,102.0,117


In [43]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
stop=set(stopwords.words('english'))
import string
list_of_rubish = ['&amp;', '-', '…', '’', '“', '—', '”', '✔', 'amp']

def create_corpus_column(tweet):
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    cleaned_corpus = []
    corpus = tokenizer.tokenize(tweet)
    for word in corpus:
        if (word not in stop) and (word not in list_of_rubish) and (word not in string.punctuation):
             cleaned_corpus.append(word)
    return cleaned_corpus

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [44]:
df['text_corpus'] = df['text'].apply(create_corpus_column)

In [45]:
df.head(5)

,username,created_at,text,lang,label,retweet_count,favorite_count,length,text_corpus
0,RepAdams,2022-03-23 18:53:00,"Madeleine Albright often said, “There is a spe...",en,D,56.0,196.0,117,"[madeleine, albright, often, said, special, pl..."
1,RepAdams,2022-03-22 14:40:16,Judge Ketanji Brown Jackson:✔️ Received a unan...,en,D,13.0,26.0,115,"[judge, ketanji, brown, jackson, ️, received, ..."
2,RepAdams,2022-03-21 20:59:02,When I Grow Up is a virtual forum designed to ...,en,D,5.0,4.0,117,"[grow, virtual, forum, designed, inspire, wome..."
3,RepAdams,2022-03-21 19:55:53,"Ketanji Brown Jackson's parents, Ellery &amp; ...",en,D,8.0,31.0,120,"[ketanji, brown, jackson's, parents, ellery, j..."
4,RepAdams,2022-03-21 13:58:54,Judge Ketanji Brown Jackson is the most qualif...,en,D,19.0,102.0,117,"[judge, ketanji, brown, jackson, qualified, su..."


In [46]:
# Unnest of text_corpus
df_tidy = df.explode(column='text_corpus')
df_tidy = df_tidy.drop(columns='text')
df_tidy.head(3)

,username,created_at,lang,label,retweet_count,favorite_count,length,text_corpus
0,RepAdams,2022-03-23 18:53:00,en,D,56.0,196.0,117,madeleine
0,RepAdams,2022-03-23 18:53:00,en,D,56.0,196.0,117,albright
0,RepAdams,2022-03-23 18:53:00,en,D,56.0,196.0,117,often


In [47]:
# Data pivot by label
df_pivot = df_tidy.groupby(["label","text_corpus"])["text_corpus"] \
                .agg(["count"]).reset_index() \
                .pivot(index = "text_corpus" , columns="label", values= "count")
df_pivot.columns.name = None

In [48]:
# Consine similarity
from scipy.spatial.distance import cosine

def similitud_coseno(a,b):
    distancia = cosine(a,b)
    return 1-distancia

df_pivot.corr(method=similitud_coseno)

,D,R
D,1.000000,0.918536
R,0.918536,1.000000


#### ***This is actually really bad news, if there is a big correlation between both groups, its gonna be really difficult to say whether a tweet tends to be republican or democrat***

---



#MODEL
#### For the feature extraction, I'm gonna use Tf-idf (Term frequency – Inverse document frequency).

In [66]:
#train-test split
from sklearn.model_selection import train_test_split

datos_X = df.text
datos_y = df.label

X_train, X_test, y_train, y_test = train_test_split(
    datos_X,
    datos_y,
    test_size = 0.2,
    random_state = 123
    
)

In [73]:
#check of proportions
value, counts = np.unique(y_train, return_counts=True)
print(dict(zip(value, 100 * counts / sum(counts))))
value, counts = np.unique(y_test, return_counts=True)
print(dict(zip(value, 100 * counts / sum(counts))))

{'D': 55.51141053977046, 'R': 44.48858946022954}
{'D': 55.40755731898916, 'R': 44.59244268101084}


In [74]:
#Tf-idf matrix
from sklearn.feature_extraction.text import TfidfVectorizer

def tokenize(tweet):
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    corpus = tokenizer.tokenize(tweet)
    return corpus

vectorizer  = TfidfVectorizer(
                        tokenizer  = tokenize,
                        min_df     = 5 #might change
                    )
vectorizer.fit(X_train)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


TfidfVectorizer(min_df=5, tokenizer=<function tokenize at 0x7f63e0e74a70>)

In [75]:
tfidf_train = vectorizer.transform(X_train)
tfidf_test  = vectorizer.transform(X_test)